<img src="https://whatcar.vn/media/2018/09/car-lot-940x470.jpg"/>

## Парсинг данных
 

In [37]:
import requests
from bs4 import BeautifulSoup 

import pandas as pd
import numpy as np

import re
import time
import json

## Парсинг данных

Source: https://auto.ru

Парсить будем только марки автомобиля, указанные в тестовом датасете

In [56]:
brands = ['bmw',
 'volkswagen',
 'nissan',
 'mercedes',
 'toyota',
 'audi',
 'mitsubishi',
 'skoda',
 'volvo',
 'honda',
 'infiniti',
 'lexus']

**Напишем функцию, которая соберёт ссылки на все машины**

In [57]:
def collect_car_urls(
    brand: str
):
    """
    По каждой марке автомобиля собирается кол-во страниц с объявлениями.
    Далее собираются ссылки на объявления в список
    
    Безбожно скопировано у Евгении Войтик
    """
    main_url = f'https://auto.ru/cars/{brand}/all/'   
    main_response = requests.get(main_url)   
    main_soap = BeautifulSoup(main_response.content.decode('utf-8'), 'html.parser')    
    _ = main_soap.find('span', class_='ButtonWithLoader__content').text.replace(u'\xa0', '')
    urls_total = int(re.findall(r'\d+', _)[0])
    ads_per_page = len(main_soap.find_all('a', class_='Link ListingItemTitle__link'))
    pages_num = urls_total // ads_per_page
    
    all_urls = []
    
    for page_num in range(1, pages_num):
        if page_num % 20 == 0:
            print(f"Extracting page {page_num} from {pages_num}...")
        page_url = f'{main_url}?page={page_num}'   
        page_response = requests.get(page_url)
        time.sleep(0.1)
        page_soap = BeautifulSoup(page_response.content.decode('utf-8'), 'html.parser')   
        
        all_urls.extend([a.get('href') for a in page_soap.find_all('a', class_='Link ListingItemTitle__link')])

    return all_urls

In [60]:
urls = []
for brand in brands:
    print(f"Extracting data for the brand {brand}:")
    all_urls = collect_car_urls(brand)
    urls.extend(all_urls)

Extracting data for the brand bmw:
Extracting page 20 from 360...
Extracting page 40 from 360...
Extracting page 60 from 360...
Extracting page 80 from 360...
Extracting page 100 from 360...
Extracting page 120 from 360...
Extracting page 140 from 360...
Extracting page 160 from 360...
Extracting page 180 from 360...
Extracting page 200 from 360...
Extracting page 220 from 360...
Extracting page 240 from 360...
Extracting page 260 from 360...
Extracting page 280 from 360...
Extracting page 300 from 360...
Extracting page 320 from 360...
Extracting page 340 from 360...
Extracting data for the brand volkswagen:
Extracting page 20 from 436...
Extracting page 40 from 436...
Extracting page 60 from 436...
Extracting page 80 from 436...
Extracting page 100 from 436...
Extracting page 120 from 436...
Extracting page 140 from 436...
Extracting page 160 from 436...
Extracting page 180 from 436...
Extracting page 200 from 436...
Extracting page 220 from 436...
Extracting page 240 from 436...
Ext

In [61]:
train_df = pd.DataFrame({'car_url': urls})
train_df.to_csv('urls.csv', index=False)

**Соберём необходимую информацию из ссылок.**

Функция:

In [323]:
def extract_url_data(url: str, extracted_columns: list):
    """
    Сбор всех признаков, представленных в тестовом датасете.
    Функция проходит по ссылке, получает информацию в текстовом формате и в json (словарь),
    далее по одному собираются признаки.
    
    каркас функции взят у Евгении Войтик, далее обновлены/добавлены методы получения необходимых признаков
    """
    response = requests.get(url)
    page = BeautifulSoup(response.content.decode('utf-8'), 'html.parser')
    
    try:
        catalog_url = page.find(
            'a', class_='Link SpoilerLink CardCatalogLink SpoilerLink_type_default').get('href')
        response_catalog = requests.get(catalog_url)
        page_catalog = BeautifulSoup(response_catalog.content.decode('utf-8'), 'html.parser')
    except:
        pass
    try:
        json_data_catalog = json.loads(
            page_catalog.find('script', type="application/json", id='initial-state').string)
    except:
        pass
    try:
        json_data_equip = json.loads(
            page.find('script', type="application/json", id='initial-state').string)
    except:
        pass
    try:
        data = json.loads(
            page.find('script', type="application/ld+json").string)
    except:
        pass
    try:
        data['bodyType'] = json_data_equip['card']['vehicle_info']['configuration']['body_type']
    except:
        pass
    try:
        data['car_url'] = url
    except:
        pass
    try:
        data['engineDisplacement'] = round(json_data_equip['card']['vehicle_info']['tech_param']['displacement'] * 0.001, 1)
    except:
        pass
    try:
        data['enginePower'] = json_data_equip['card']['vehicle_info']['tech_param']['power']
    except:
        pass
    try:
        data['fuelType'] = json_data_equip['card']['vehicle_info']['tech_param']['engine_type']
    except:
        pass
    try:
        data['modelDate'] = data['productionDate']
    except:
        pass
    try:
        data['numberOfDoors'] = json_data_equip['card']['vehicle_info']['configuration']['doors_count']
    except:
        pass
    try:
        data['complectation_dict'] = [
        k for k, v in json_data_catalog['state']['compare']['selected'][0]['options'].items() if v == 1]
    except:
        pass
    try:
        data['equipment_dict'] = json_data_equip['card']['vehicle_info']['equipment']
    except:
        pass
    try:
        mileage = page.find(
            'li', class_='CardInfoRow CardInfoRow_kmAge').find_all('span')[-1].text.replace(u'\xa0', u'')
        data['mileage'] = int(re.findall(r'\d+', mileage)[0])
    except:
        pass
    try:
        data['model_name'] = page.find_all(
        'div', class_='InfoPopup InfoPopup_theme_plain InfoPopup_withChildren BreadcrumbsPopup')[1].text
    except:
        pass
    try:
        data['parsing_unixtime'] = int(time.time())
    except:
        pass
    try:
        data['sell_id'] = int(re.findall(
            r'\d+', page.find('div', class_='CardHead__infoItem CardHead__id').text)[0])
    except:
        pass
    try:
        data['super_gen'] = json.loads(
            page.find('div', id="sale-data-attributes").get('data-bem'))
    except:
        pass
    try:
        data['vehicleTransmission'] = json_data_equip['card']['vehicle_info']['tech_param']['transmission']
    except:
        pass
    try:
        data['vendor'] = json_data_equip['card']['vehicle_info']['vendor']
    except:
        pass
    
    try:
        data['Владельцы'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_ownersCount').find_all('span')[-1].text.replace(u'\xa0', u' ')
    except:
        pass
    try:
        data['Владение'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_owningTime').find_all('span')[-1].text
    except:
        pass
    try:
        data['ПТС'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_pts').find_all('span')[-1].text
    except:
        pass
    
    try:
        data['Привод'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_drive').find_all('span')[-1].text
    except:
        pass
    
    try:
        data['Руль'] = page.find('li', class_='CardInfoRow CardInfoRow_wheel').find_all('span')[-1].text
    except:
        pass
    try:
        data['Состояние'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_state').find_all('span')[-1].text
    except:
        pass
    try:
        data['Таможня'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_customs').find_all('span')[-1].text
    except:
        pass
    try:
        data['description'] = re.sub('\W+', ' ', data['description'])
    except:
        pass
    
    # additional features
    try:
        data['price'] = page.find(
            'span', class_='OfferPriceCaption__price').text.replace('\xa0', '').replace('₽', '')
    except:
        pass
    
    output = []
    try:
        for col in extracted_columns:
            output.append(data.get(col, None))    
    except:
        pass
    if not output:
        output = [None] * len(extracted_columns)
    return output

In [68]:
test_cols = test.columns.to_list()

In [295]:
df_combined = pd.read_csv('urls.csv')
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133495 entries, 0 to 133494
Data columns (total 1 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   car_url  133495 non-null  object
dtypes: object(1)
memory usage: 1.0+ MB


In [296]:
final_df = pd.DataFrame()

In [321]:
import time

start = time.time()


final_list = []
for n, url in enumerate(df_combined.car_url.values.tolist()[1001:]):
    if n % 50 == 0:
        print(f"The # of current processing URL is {n}, url is {url}.")
    if n % 1000 == 0:
        print(f"Create a dataframe from the list and concetenate it with the already collected data.")
        now = time.time()
        print(f'Time from the start: {round((now-start)/60)} minutes')
        
        df1 = pd.DataFrame(data=final_list, columns=test_cols + ['views', 'date_added', 'region', 'price'])
        final_df = pd.concat([final_df, df1], ignore_index=True)
        final_df.to_csv('train_df_full_part2.csv', index=False)
        final_list = []
    final_list.append(extract_url_data(url, test_cols + ['views', 'date_added', 'region', 'price']))

The # of current processing URL is 0, url is https://auto.ru/cars/used/sale/bmw/x7/1106101849-d37d03b1/.
Create a dataframe from the list and concetenate it with the already collected data.
Time from the start: 0 minutes
The # of current processing URL is 50, url is https://auto.ru/cars/used/sale/bmw/1er/1105978255-dd709cad/.
The # of current processing URL is 100, url is https://auto.ru/cars/used/sale/bmw/x3/1105862336-eae6db74/.
The # of current processing URL is 150, url is https://auto.ru/cars/used/sale/bmw/6er/1105454822-24cd5a63/.
The # of current processing URL is 200, url is https://auto.ru/cars/used/sale/bmw/x4_m/1106087910-a17ef445/.
The # of current processing URL is 250, url is https://auto.ru/cars/used/sale/bmw/3er/1105756745-b6f97aef/.
The # of current processing URL is 300, url is https://auto.ru/cars/new/group/bmw/x6/21611056/21761005/1106065395-ab2dfa60/.
The # of current processing URL is 350, url is https://auto.ru/cars/used/sale/bmw/x1/1105733404-ce056cea/.
The # of

The # of current processing URL is 3450, url is https://auto.ru/cars/used/sale/bmw/5er/1105568620-20668389/.
The # of current processing URL is 3500, url is https://auto.ru/cars/new/group/bmw/x5/21308274/21640936/1105761618-b8ab9cb1/.
The # of current processing URL is 3550, url is https://auto.ru/cars/used/sale/bmw/x6_m/1105787740-dfc4e8ed/.
The # of current processing URL is 3600, url is https://auto.ru/cars/used/sale/bmw/7er/1105722944-62e663cb/.
The # of current processing URL is 3650, url is https://auto.ru/cars/used/sale/bmw/x6/1105611363-34b4c988/.
The # of current processing URL is 3700, url is https://auto.ru/cars/used/sale/bmw/x1/1106171414-15939271/.
The # of current processing URL is 3750, url is https://auto.ru/cars/used/sale/bmw/x5/1104640280-4bf7777d/.
The # of current processing URL is 3800, url is https://auto.ru/cars/used/sale/bmw/7er/1104813489-ea20dcef/.
The # of current processing URL is 3850, url is https://auto.ru/cars/used/sale/bmw/5er/1105568620-20668389/.
The 

The # of current processing URL is 7050, url is https://auto.ru/cars/used/sale/bmw/z4/1106003617-47d5f34d/.
The # of current processing URL is 7100, url is https://auto.ru/cars/new/group/bmw/x5/21308274/21640936/1105816739-c4e67f63/.
The # of current processing URL is 7150, url is https://auto.ru/cars/used/sale/bmw/7er/1105722944-62e663cb/.
The # of current processing URL is 7200, url is https://auto.ru/cars/used/sale/bmw/3er/1105789589-5fa4d19f/.
The # of current processing URL is 7250, url is https://auto.ru/cars/used/sale/bmw/7er/1104813489-ea20dcef/.
The # of current processing URL is 7300, url is https://auto.ru/cars/used/sale/bmw/3er/1105827427-62f87f75/.
The # of current processing URL is 7350, url is https://auto.ru/cars/used/sale/bmw/x4/1106119271-46f0ffd7/.
The # of current processing URL is 7400, url is https://auto.ru/cars/used/sale/bmw/7er/1105722944-62e663cb/.
The # of current processing URL is 7450, url is https://auto.ru/cars/used/sale/bmw/x6/1105611363-34b4c988/.
The #

The # of current processing URL is 10650, url is https://auto.ru/cars/used/sale/bmw/3er/1106172036-44fc1f65/.
The # of current processing URL is 10700, url is https://auto.ru/cars/used/sale/bmw/x3/1105433797-66be98ed/.
The # of current processing URL is 10750, url is https://auto.ru/cars/new/group/bmw/x5/21308274/21640936/1105816739-c4e67f63/.
The # of current processing URL is 10800, url is https://auto.ru/cars/used/sale/bmw/x7/1105987256-bae06da7/.
The # of current processing URL is 10850, url is https://auto.ru/cars/new/group/bmw/x5/21308274/21640936/1106025849-39d87da2/.
The # of current processing URL is 10900, url is https://auto.ru/cars/used/sale/bmw/x6/1105611363-34b4c988/.
The # of current processing URL is 10950, url is https://auto.ru/cars/used/sale/bmw/x6/1105611363-34b4c988/.
The # of current processing URL is 11000, url is https://auto.ru/cars/used/sale/bmw/x6/1106171437-bcd527e6/.
Create a dataframe from the list and concetenate it with the already collected data.
Time f

The # of current processing URL is 14050, url is https://auto.ru/cars/used/sale/volkswagen/tiguan/1106140648-25497ba9/.
The # of current processing URL is 14100, url is https://auto.ru/cars/used/sale/volkswagen/passat/1105794395-d3ff762a/.
The # of current processing URL is 14150, url is https://auto.ru/cars/used/sale/volkswagen/polo/1105785776-34cf6488/.
The # of current processing URL is 14200, url is https://auto.ru/cars/used/sale/volkswagen/touareg/1106136845-ef4cf33e/.
The # of current processing URL is 14250, url is https://auto.ru/cars/used/sale/volkswagen/tiguan/1106135237-9095dbc5/.
The # of current processing URL is 14300, url is https://auto.ru/cars/used/sale/volkswagen/touareg/1105488274-555ac61c/.
The # of current processing URL is 14350, url is https://auto.ru/cars/used/sale/volkswagen/polo/1106129975-9cdf4e2b/.
The # of current processing URL is 14400, url is https://auto.ru/cars/used/sale/volkswagen/teramont/1106042327-3ff43ef3/.
The # of current processing URL is 14450

...

The # of current processing URL is 117050, url is https://auto.ru/cars/used/sale/skoda/fabia/1105632977-61215ecb/.
The # of current processing URL is 117100, url is https://auto.ru/cars/used/sale/skoda/fabia/1105610286-9c5fdfab/.
The # of current processing URL is 117150, url is https://auto.ru/cars/used/sale/skoda/rapid/1106051827-6bdab8c2/.
The # of current processing URL is 117200, url is https://auto.ru/cars/used/sale/skoda/rapid/1105594754-592e06dc/.
The # of current processing URL is 117250, url is https://auto.ru/cars/used/sale/skoda/octavia/1105505881-5b441305/.
The # of current processing URL is 117300, url is https://auto.ru/cars/new/group/skoda/rapid/21738489/22128588/1105638449-02d5100d/.
The # of current processing URL is 117350, url is https://auto.ru/cars/used/sale/skoda/yeti/1105880732-7c69d2b3/.
The # of current processing URL is 117400, url is https://auto.ru/cars/used/sale/skoda/octavia/1106160166-f1693c7d/.
The # of current processing URL is 117450, url is https://a

The # of current processing URL is 120300, url is https://auto.ru/cars/new/group/volvo/xc60/21158358/22174038/1105341194-55e3a0e6/.
The # of current processing URL is 120350, url is https://auto.ru/cars/used/sale/volvo/xc60/1105872267-795f1c8c/.
The # of current processing URL is 120400, url is https://auto.ru/cars/used/sale/volvo/xc60/1105838231-564f3ee5/.
The # of current processing URL is 120450, url is https://auto.ru/cars/new/group/volvo/xc90/21553368/21555601/1105901242-8c0d8d56/.
The # of current processing URL is 120500, url is https://auto.ru/cars/new/group/volvo/xc60/21158358/22174038/1105228422-a18aaa8f/.
The # of current processing URL is 120550, url is https://auto.ru/cars/new/group/volvo/xc60/22991821/22992212/1105823088-4fc08839/.
The # of current processing URL is 120600, url is https://auto.ru/cars/new/group/volvo/xc40/22175484/23000177/1105817217-d3485583/.
The # of current processing URL is 120650, url is https://auto.ru/cars/used/sale/volvo/xc90/1105972769-600c3793/

The # of current processing URL is 123600, url is https://auto.ru/cars/used/sale/honda/stepwagon/1105942741-b750fb16/.
The # of current processing URL is 123650, url is https://auto.ru/cars/used/sale/honda/accord/1105859427-6aab74bb/.
The # of current processing URL is 123700, url is https://auto.ru/cars/used/sale/honda/orthia/1105177658-01d4b4d0/.
The # of current processing URL is 123750, url is https://auto.ru/cars/used/sale/honda/stepwagon/1105780544-73f586c1/.
The # of current processing URL is 123800, url is https://auto.ru/cars/used/sale/honda/freed/1104938433-f21f1e9a/.
The # of current processing URL is 123850, url is https://auto.ru/cars/used/sale/honda/odyssey/1105375333-e6e72e74/.
The # of current processing URL is 123900, url is https://auto.ru/cars/used/sale/honda/stream/1106106835-ce75a169/.
The # of current processing URL is 123950, url is https://auto.ru/cars/used/sale/honda/stepwagon/1105704107-d3c17b0f/.
The # of current processing URL is 124000, url is https://auto.

The # of current processing URL is 127000, url is https://auto.ru/cars/used/sale/infiniti/m/1105423980-9ce3f20e/.
Create a dataframe from the list and concetenate it with the already collected data.
Time from the start: 3186 minutes
The # of current processing URL is 127050, url is https://auto.ru/cars/used/sale/infiniti/g35/1106094195-306f796a/.
The # of current processing URL is 127100, url is https://auto.ru/cars/used/sale/infiniti/q50/1106066009-290b305f/.
The # of current processing URL is 127150, url is https://auto.ru/cars/used/sale/infiniti/qx56/1104992666-13f68ef5/.
The # of current processing URL is 127200, url is https://auto.ru/cars/used/sale/infiniti/qx56/1105524467-71829d8e/.
The # of current processing URL is 127250, url is https://auto.ru/cars/used/sale/infiniti/qx70/1105939096-6b7c70a5/.
The # of current processing URL is 127300, url is https://auto.ru/cars/used/sale/infiniti/fx/1106147175-ed6f6c4b/.
The # of current processing URL is 127350, url is https://auto.ru/car

The # of current processing URL is 130300, url is https://auto.ru/cars/used/sale/lexus/rx/1106044722-a2879270/.
The # of current processing URL is 130350, url is https://auto.ru/cars/new/group/lexus/rx/21662860/21663185/1105965714-0e075c91/.
The # of current processing URL is 130400, url is https://auto.ru/cars/used/sale/lexus/gx/1106078667-19a72a93/.
The # of current processing URL is 130450, url is https://auto.ru/cars/new/group/lexus/rx/21662860/21663185/1105616962-2621d516/.
The # of current processing URL is 130500, url is https://auto.ru/cars/used/sale/lexus/ux/1104832270-2f0a87e0/.
The # of current processing URL is 130550, url is https://auto.ru/cars/new/group/lexus/rx/21662860/21663007/1106164047-12998804/.
The # of current processing URL is 130600, url is https://auto.ru/cars/used/sale/lexus/nx/1105641327-47feb158/.
The # of current processing URL is 130650, url is https://auto.ru/cars/new/group/lexus/rx/21662860/21663185/1105819068-767e030f/.
The # of current processing URL 

In [322]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133000 entries, 0 to 132999
Data columns (total 36 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   bodyType              132522 non-null  object 
 1   brand                 132536 non-null  object 
 2   car_url               132969 non-null  object 
 3   color                 132536 non-null  object 
 4   complectation_dict    100874 non-null  object 
 5   description           132536 non-null  object 
 6   engineDisplacement    132522 non-null  float64
 7   enginePower           132521 non-null  float64
 8   equipment_dict        132522 non-null  object 
 9   fuelType              132521 non-null  object 
 10  image                 132852 non-null  object 
 11  mileage               101342 non-null  float64
 12  modelDate             132536 non-null  float64
 13  model_info            0 non-null       object 
 14  model_name            101342 non-null  object 
 15  

In [73]:
len(urls)

133495